In [1]:
import pandas as pd
import numpy as np
import pydicom 
import os
import matplotlib.pyplot as plt

from sklearn.model_selection import train_test_split

from keras.models import Model, Sequential
from keras.layers import Concatenate, Dense, Input, concatenate
from keras.layers import Dense, Dropout, Conv2D, MaxPool2D, Flatten

Using TensorFlow backend.


In [2]:
def load_best_slices(path):
    ind = []
    for file in os.listdir(path):
        if file.split('.')[1] == 'txt':
            ind.append(file.split('.')[0])
        
    df = pd.DataFrame(index = ind, columns= ['CT'])
    for ind in df.index:
        df.loc[ind].CT = np.loadtxt(path + ind + '.txt')
    
    return df

df = load_best_slices('best_lung_slice/')

In [3]:
linear_data_all = pd.read_csv('patient_slope_intercept.csv', index_col=0)
result = pd.DataFrame(index = df.index, columns = ['slope', 'intersect'])

for ind in result.index:
    result.loc[ind].slope = linear_data_all.loc[ind].slope
    result.loc[ind].intersect = linear_data_all.loc[ind].intersect

In [4]:
dataset = df.values

In [5]:
X = []
for ind in df.index:
    X.append(df.loc[ind].CT)

X = np.array(X)
X = X.reshape(X.shape[0], 512, 512, 1)

In [7]:
from sklearn.preprocessing import MinMaxScaler
y = result.values
scaler = MinMaxScaler()
scaler.fit(y)
y_scaled = scaler.transform(y)

In [23]:
X_train, X_test, y_train, y_test = train_test_split(X, y_scaled, test_size=0.3)

In [24]:
# building a linear stack of layers with the sequential model
model = Sequential()
# convolutional layer
model.add(Conv2D(12, kernel_size=(3,3), strides=(1,1), padding='valid', activation='relu', input_shape=(512,512,1)))
model.add(MaxPool2D(pool_size=(2,2)))
model.add(Dropout(0.25))

model.add(Conv2D(8, kernel_size=(3,3), strides=(1,1), padding='valid', activation='relu'))
model.add(MaxPool2D(pool_size=(3,3)))
model.add(Dropout(0.25))

# flatten output of conv
model.add(Flatten())
# hidden layer
model.add(Dense(10, activation='relu'))
model.add(Dropout(0.25))
model.add(Dense(5, activation='relu'))
# output layer
model.add(Dense(2, activation='sigmoid'))

In [25]:
model.compile(loss='mean_squared_error', metrics=['mean_squared_error'], optimizer='adam')

In [27]:
model.fit(X_train,y_train, batch_size=32, epochs=20, validation_data=(X_test,y_test))

Train on 123 samples, validate on 53 samples
Epoch 1/20
123/123 [==============================] - 9s 73ms/step - loss: 0.0442 - mean_squared_error: 0.0442 - val_loss: 0.0347 - val_mean_squared_error: 0.0347
Epoch 2/20
123/123 [==============================] - 10s 79ms/step - loss: 0.0334 - mean_squared_error: 0.0334 - val_loss: 0.0355 - val_mean_squared_error: 0.0355
Epoch 3/20
123/123 [==============================] - 11s 87ms/step - loss: 0.0315 - mean_squared_error: 0.0315 - val_loss: 0.0345 - val_mean_squared_error: 0.0345
Epoch 4/20
123/123 [==============================] - 10s 82ms/step - loss: 0.0286 - mean_squared_error: 0.0286 - val_loss: 0.0341 - val_mean_squared_error: 0.0341
Epoch 5/20
123/123 [==============================] - 9s 77ms/step - loss: 0.0282 - mean_squared_error: 0.0282 - val_loss: 0.0340 - val_mean_squared_error: 0.0340
Epoch 6/20
123/123 [==============================] - 10s 79ms/step - loss: 0.0281 - mean_squared_error: 0.0281 - val_loss: 0.0342 - val_

In [28]:
scaler.inverse_transform(model.predict(X_test))

array([[  -5.4439683, 3267.4197   ],
       [  -4.8745356, 3235.5615   ],
       [  -5.48275  , 3269.588    ],
       [  -6.945752 , 3360.7368   ],
       [  -5.1827054, 3252.808    ],
       [  -6.722429 , 3338.9028   ],
       [  -5.392951 , 3264.5671   ],
       [  -5.701289 , 3281.8054   ],
       [  -3.463889 , 3156.3064   ],
       [  -5.911453 , 3293.553    ],
       [  -4.9355636, 3238.978    ],
       [  -6.4390783, 3323.0513   ],
       [  -6.7118974, 3338.3137   ],
       [  -5.8105803, 3287.9146   ],
       [  -6.8063674, 3343.6008   ],
       [  -5.918558 , 3293.9504   ],
       [  -6.554297 , 3329.4958   ],
       [  -6.843404 , 3348.0454   ],
       [  -4.362986 , 3206.8918   ],
       [  -6.945752 , 3360.7368   ],
       [  -6.945752 , 3360.7368   ],
       [  -6.945752 , 3360.7368   ],
       [  -6.832664 , 3346.714    ],
       [  -6.825987 , 3345.8865   ],
       [  -6.3724627, 3319.3262   ],
       [  -4.9745603, 3241.161    ],
       [  -6.945752 , 3360.7368   ],
 

In [18]:
n=8
example = df.iloc[n].CT.reshape(1,512,512,1)
print(scaler.inverse_transform(model.predict(example)))
print(result.iloc[n])

[[  -4.6193943 3287.677    ]]
slope       -3.41031
intersect    3636.58
Name: ID00229637202260254240583, dtype: object


In [14]:
scaler.inverse_transform(model.predict(X_test)) - y_test

array([[  -5.10545461, 3307.48428664],
       [  -4.78305531, 3308.12438375],
       [  -4.89725229, 3307.83212794],
       [  -5.22165662, 3307.8550803 ],
       [  -5.42184356, 3307.78302211],
       [  -5.43281351, 3308.12617764],
       [  -5.16927023, 3307.59642907],
       [  -5.24072548, 3308.16093316],
       [  -5.35191159, 3308.08266831],
       [  -5.33692126, 3307.92509442],
       [  -5.26648015, 3308.33170063],
       [  -5.31021219, 3308.13071862],
       [  -5.3362097 , 3307.95688147],
       [  -5.28324564, 3307.8856003 ],
       [  -5.32835758, 3307.98839754],
       [  -5.44998669, 3308.09456244],
       [  -5.31234745, 3308.01621194],
       [  -5.24865659, 3308.08910545],
       [  -5.4360549 , 3307.89741209],
       [  -5.18908541, 3307.96594882],
       [  -5.29321019, 3308.04920766],
       [  -5.36017272, 3307.81951334],
       [  -5.53804201, 3307.92597347],
       [  -5.48692184, 3307.99768183],
       [  -5.23259914, 3308.06525225],
       [  -5.65170388, 33

In [15]:
y_test

array([[0.3223993 , 0.88680711],
       [0.        , 0.24671   ],
       [0.11419698, 0.53896581],
       [0.43860131, 0.51601345],
       [0.63878826, 0.58807164],
       [0.6497582 , 0.24491611],
       [0.38621492, 0.77466468],
       [0.45767017, 0.21016059],
       [0.56885628, 0.28842544],
       [0.55386596, 0.44599933],
       [0.48342484, 0.03939312],
       [0.52715689, 0.24037513],
       [0.5531544 , 0.41421228],
       [0.50019034, 0.48549345],
       [0.54530227, 0.38269621],
       [0.66693138, 0.27653131],
       [0.52929214, 0.35488181],
       [0.46560128, 0.2819883 ],
       [0.6529996 , 0.47368166],
       [0.40603011, 0.40514493],
       [0.51015489, 0.32188609],
       [0.57711742, 0.55158041],
       [0.75498671, 0.44512028],
       [0.70386654, 0.37341192],
       [0.44954384, 0.3058415 ],
       [0.86864858, 0.17456617],
       [0.67365498, 0.36069631],
       [0.22864234, 0.31243976],
       [0.40478145, 0.46521451],
       [0.64420815, 0.37470444],
       [0.